In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

Mounted at /content/drive


In [ ]:
import torch
TORCH = torch.__version__.split('+')[0]
CUDA = 'cu' + torch.version.cuda.replace('.','')

import pickle
import os

In [ ]:
%%capture
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install node2vec
!pip install 'scipy>=1.8'
!pip install mycolorpy
!git clone https://github.com/ellisalicante/GraphRewiring-Tutorialz
!cd GraphRewiring-Tutorial && git submodule update --init --recursive
!mv GraphRewiring-Tutorial/* ./
!rm -rf GraphRewiring-Tutorial
!pip install Cython
!pip install Ripser
!pip install -U giotto-tda
!pip install rdkit

In [ ]:
import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

import torchvision
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from scipy.linalg import fractional_matrix_power
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

import matplotlib.pyplot as plt
import matplotlib as matplotlib
import matplotlib.cm as cm
from mycolorpy import colorlist as mcp

from tqdm import tqdm

from ripser import ripser
from persim import plot_diagrams
import persim
import copy

from gtda.homology import VietorisRipsPersistence,SparseRipsPersistence
from gtda.diagrams import PairwiseDistance


import math
from numba import cuda
import numpy as np
import torch
from torch_geometric.utils import (
    to_networkx,
    from_networkx,
    to_dense_adj,
    remove_self_loops,
    to_undirected,
)

In [ ]:
import sys
import os
sys.path.insert(0, "/content/drive/MyDrive/CubeRep-Rewiring/src")

In [ ]:
%%capture
!pip install POT

from fosr import *
from gtr import *
from relwire import *
from sdrf import *
from tmd import *
from utils import *

### Load Data

In [ ]:
## Standard version for everyone
# dataset = torch_geometric.datasets.TUDataset(root="data/",name = "IMDB-BINARY")
dataset = torch_geometric.datasets.TUDataset(root="data/",name = "COLLAB")
# dataset = torch_geometric.datasets.TUDataset(root="data/",name = "REDDIT-BINARY")
# dataset = torch_geometric.datasets.TUDataset(root="data/",name = "PROTEINS")
# dataset = torch_geometric.datasets.TUDataset(root="data/",name = "MUTAG")
# dataset = torch_geometric.datasets.TUDataset(root="data/",name = "ENZYMES")

# dataset = torch_geometric.datasets.MoleculeNet(root="data/",name = "BACE")
# dataset = torch_geometric.datasets.MoleculeNet(root="data/",name = "ESOL")
# dataset = torch_geometric.datasets.MoleculeNet(root="data/",name = "Lipo")

# dataset = torch_geometric.datasets.ZINC(root="data/", subset = True)


# dataset = torch_geometric.datasets.LRGBDataset(root="data/",name = "PascalVOC-SP")
# dataset = torch_geometric.datasets.LRGBDataset(root="data/",name = "COCO-SP")
# dataset = torch_geometric.datasets.LRGBDataset(root="data/",name = "PCQM-Contact)
# dataset = torch_geometric.datasets.LRGBDataset(root="data/",name = "Peptides-func")
# dataset = torch_geometric.datasets.LRGBDataset(root="data/",name = "Peptides-struct")

In [ ]:
# Gs = []
# for i in tqdm(range(len(dataset))):
#   Gs.append(dataset[i].edge_index)

# # with open('/content/drive/MyDrive/CubeRep-Rewiring/data/collab-edge-index.pkl', "wb") as f:
# #   pickle.dump(Gs,f,protocol=pickle.HIGHEST_PROTOCOL)

# torch.save(Gs, '/content/drive/MyDrive/CubeRep-Rewiring/data/imdb-binary-edge-index.pt')

100%|██████████| 1000/1000 [00:00<00:00, 12325.20it/s]


In [ ]:
## GTR Version
import torch_geometric.transforms as T
pre_transform = T.Compose([PrecomputeGTREdges(num_edges=20)])
transform = T.Compose([AddPrecomputedGTREdges(num_edges=20)])
dataset_gtr = torch_geometric.datasets.TUDataset(root="data/",name = "IMDB-BINARY", transform = transform, pre_transform=pre_transform)

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/dataset.py:214: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, make sure to delete 'data/IMDB-BINARY/processed' first
  warnings.warn(


In [ ]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: COLLAB(5000):
Number of graphs: 5000
Number of features: 0
Number of classes: 3

Data(edge_index=[2, 1980], y=[1], num_nodes=45)
Number of nodes: 45
Number of edges: 1980
Average node degree: 44.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [ ]:
def rewire(dataset, rewiring, k):
  N = len(dataset)
  dataset_new = []
  for i in tqdm(range(N)):
    data = dataset[i]
    G = nx.from_numpy_array(to_dense_adj(data.edge_index)[0].numpy())
    empty = False
    if G.number_of_edges() < 0.5:
      empty = True
    elif rewiring == "HHS":
      A, edge_to_index, L, D = create_views(G)
      sorted_counts = get_sorted_counts(G,k,A,edge_to_index,L,D,dist=True)
      G2 = rewire_hhs(copy.deepcopy(G),k,A,edge_to_index,L,D,sorted_counts,True)
    elif rewiring == "sdrf":
      G2 = sdrf(copy.deepcopy(G),loops=k)
    else:
      # G2 = fosr(copy.deepcopy(G),r=50, k=k)
      edge_index,_,_ = edge_rewire(copy.deepcopy(data.edge_index.numpy()),initial_power_iters=50, num_iterations=k)
      G2 = nx.from_numpy_array(to_dense_adj(torch.tensor(edge_index))[0].numpy())

    if not empty:
      data.rewired_edge_index = torch.tensor(nx.adjacency_matrix(G2).toarray()).to(torch.float).nonzero().t().contiguous()
    else:
      data.rewired_edge_index = data.edge_index

    dataset_new.append(data)
  return dataset_new

def merge_datasets(data_x, data_edge):
  dataset_new = []
  N = len(data_x)
  for i in tqdm(range(N)):
    data = data_x[i]
    data.rewired_edge_index = data_edge[i].rewired_edge_index
    dataset_new.append(data)
  return dataset_new

def merge_datasets_from_adj(data_x, data_edge):
  dataset_new = []
  N = len(data_x)
  for i in tqdm(range(N)):
    data = data_x[i]
    data.rewired_edge_index = adj_to_edge(data_edge[i])
    dataset_new.append(data)
  return dataset_new

class RewiredDataset(InMemoryDataset):
  def __init__(self, dataset, rewiring, k, merge = False, data_edge = None):
    super(RewiredDataset, self).__init__('.', None, None, None)
    if merge:
      self.dataset_rewired = merge_datasets(dataset, data_edge)
    else:
      self.dataset_rewired = rewire(dataset, rewiring, k)

  def get(self, idx):
    return self.dataset_rewired[idx]

  def len(self):
    return len(self.dataset_rewired)

  def _download(self):
    return

  def _process(self):
      return

  def __repr__(self):
      return '{}()'.format(self.__class__.__name__)

In [ ]:
hhs_dataset = RewiredDataset(dataset, "HHS", 5)
torch.save(hhs_dataset, "/content/drive/MyDrive/CubeRep-Rewiring/data/imdb-binary-hhs.pt")

  0%|          | 0/10873 [00:24<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
for k in range(1,21):
  fosr_dataset = RewiredDataset(dataset, "fosr", k)
  torch.save(fosr_dataset, "/content/drive/MyDrive/CubeRep-Rewiring/data/peptides-struct-fosr-"+str(k)+".pt")

  0%|          | 0/10873 [00:00<?, ?it/s]<ipython-input-17-419745086e98>:22: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  data.rewired_edge_index = torch.tensor(nx.adjacency_matrix(G2).toarray()).to(torch.float).nonzero().t().contiguous()
100%|██████████| 10873/10873 [01:05<00:00, 166.78it/s]


In [ ]:
for k in range(1,21):
  sdrf_dataset = RewiredDataset(dataset, "sdrf", k)
  torch.save(sdrf_dataset, "/content/drive/MyDrive/CubeRep-Rewiring/data/peptides-struct-sdrf-"+str(k)+".pt")

  0%|          | 0/10873 [00:00<?, ?it/s]/content/drive/MyDrive/CubeRep-Rewiring/src/sdrf.py:211: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)
/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
<ipython-input-17-419745086e98>:22: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  data.rewired_edge_index = torch.tensor(nx.adjacency_matrix(G2).toarray()).to(torch.float).nonzero().t().contiguous()
  0%|          | 1/10873 [00:00<1:05:57,  2.75it/s]/usr/local/lib/python3.10/dist-packages/

# Calculate metrics

In [ ]:
def edge_to_graph(edge_index):
  adj_matrix = to_dense_adj(edge_index)[0]
  G = nx.from_numpy_array(adj_matrix.numpy())
  return G

def graph_to_edge(G):
  return torch.tensor(nx.adjacency_matrix(G).toarray()).to(torch.float).nonzero().t().contiguous()

def adj_to_edge(A):
  return A.to(torch.float).nonzero().t().contiguous()

def calc_eigengap(G):
  LG = nx.normalized_laplacian_matrix(G)
  eLG, evecsLG = find_eigen(LG.todense())
  gapLG = eLG[1]
  return gapLG

def calc_average_commute(G):
  return commute_times(G).mean()

def calc_average_curvature(G):
  return curvature(G)[0].mean()

def calc_persistence(G):
  n = G.number_of_nodes()
  d = nx.algorithms.shortest_paths.dense.floyd_warshall(G)
  D = torch.zeros(n,n, dtype = torch.double)
  for i in range(n):
    for j in range(n):
      D[i,j] = d[i][j]

  diagrams = ripser(D.numpy(), distance_matrix = True)['dgms']
  return diagrams

def calc_bottleneck_distance(G_orig, G_new):
  diagrams_orig = calc_persistence(G_orig)
  diagrams_new = calc_persistence(G_new)
  try:
    distance_bottleneck, matching = persim.bottleneck(diagrams_orig[1], diagrams_new[1], matching=True)
  except:
    distance_bottleneck = 0
  return distance_bottleneck

def graphs_to_dists(Gs):
  n = Gs[0].number_of_nodes()
  N = len(Gs)
  Ds = torch.zeros(N,n,n, dtype = torch.double)
  for idx in range(N):
    d = nx.algorithms.shortest_paths.dense.floyd_warshall(Gs[idx])
    for i in range(n):
      for j in range(n):
        Ds[idx,i,j] = d[i][j]
  return Ds

def compute_homologies(Ds, dim = 8, sparse = True):
  if sparse:
    VR = SparseRipsPersistence(metric = "precomputed", homology_dimensions=list(range(1,dim+1)))
  else:
    VR = VietorisRipsPersistence(metric = "precomputed", homology_dimensions=list(range(1,dim+1)))
  return VR.fit_transform(Ds)

def homology_distance(dgms, metric = "wasserstein"):
  hom_dist = PairwiseDistance(metric = metric)
  return hom_dist.fit_transform(dgms)

In [ ]:
import torch_geometric.transforms as T
pre_transform = T.Compose([PrecomputeGTREdges(num_edges=1000)])
transform = T.Compose([AddPrecomputedGTREdges(num_edges=1000)])
dataset_gtr = torch_geometric.datasets.Planetoid(root="data/",name = "CiteSeer", transform = transform, pre_transform=pre_transform)

PreGTR = PrecomputeGTREdges(num_edges=1000)
Gs_gtr = []
G_data = Data(edge_index = dataset.edge_index)
G_data_pre = PreGTR(G_data)
for i in tqdm(range(1000)):
  GTR = AddPrecomputedGTREdges(num_edges=i+1)
  G_gtr = GTR(copy.deepcopy(G_data_pre)).edge_index
  Gs_gtr.append(G_gtr)

In [ ]:
# dataset_gtr = torch_geometric.datasets.MoleculeNet(root="data_rewired/", name = "Lipo", transform = transform, pre_transform=pre_transform)
# dataset = torch_geometric.datasets.MoleculeNet(root="data/", name = "Lipo")

dataset_gtr = torch_geometric.datasets.ZINC(root="data_rewired/", subset = True, split = "train", transform = transform, pre_transform=pre_transform)
dataset = torch_geometric.datasets.ZINC(root="data/", subset = True, split = "train")

In [ ]:
# hhs_dataset = RewiredDataset(dataset, "HHS", 3)
# torch.save(hhs_dataset, "/content/drive/MyDrive/CubeRep-Rewiring/data/LINUX-hhs.pt")
hhs_dataset = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/data/zinc-hhs.pt")

In [ ]:
# fosr_dataset = RewiredDataset(dataset, "fosr", 3)
# torch.save(fosr_dataset, "/content/drive/MyDrive/CubeRep-Rewiring/data/LINUX-fosr.pt")
fosr_dataset = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/data/zinc-fosr.pt")

In [ ]:
# sdrf_dataset = RewiredDataset(dataset, "sdrf", 3)
# torch.save(sdrf_dataset, "/content/drive/MyDrive/CubeRep-Rewiring/data/LINUX-sdrf.pt")
sdrf_dataset = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/data/zinc-sdrf.pt")

In [ ]:
eigen_gap = torch.zeros(5,200)
average_commute_times = torch.zeros(5,200)
average_curvature = torch.zeros(5,200)
bottleneck_distance = torch.zeros(4,200)
tmd_distance = torch.zeros(4,200)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

for i in tqdm(range(200)):
  G_orig = edge_to_graph(dataset[i].edge_index)
  G_hhs = edge_to_graph(hhs_dataset[i].rewired_edge_index)
  G_sdrf = edge_to_graph(sdrf_dataset[i].rewired_edge_index)
  G_fosr = edge_to_graph(fosr_dataset[i].rewired_edge_index)
  G_gtr = edge_to_graph(dataset_gtr[i].edge_index)


  G_hhs_data = Data(x = hhs_dataset[i].x.to(torch.float), edge_index = hhs_dataset[i].rewired_edge_index)
  G_sdrf_data = Data(x = hhs_dataset[i].x.to(torch.float), edge_index = sdrf_dataset[i].rewired_edge_index)
  G_fosr_data = Data(x = hhs_dataset[i].x.to(torch.float), edge_index = fosr_dataset[i].rewired_edge_index)
  G_orig_data = Data(x = hhs_dataset[i].x.to(torch.float), edge_index = dataset[i].edge_index)
  G_gtr_data = Data(x = hhs_dataset[i].x.to(torch.float), edge_index = dataset_gtr[i].edge_index)

  # print(G_orig.number_of_nodes(), G_hhs.number_of_nodes())

  if G_orig.number_of_edges() > 0:

    try:
      eigen_gap[0,i] = calc_eigengap(G_orig)
    except:
      print(calc_eigengap(G_orig))
    try:
      eigen_gap[1,i] = calc_eigengap(G_hhs)
    except:
      print(calc_eigengap(G_hhs))
    try:
      eigen_gap[2,i] = calc_eigengap(G_sdrf)
    except:
      print(calc_eigengap(G_sdrf))
    try:
      eigen_gap[3,i] = calc_eigengap(G_fosr)
    except:
      print(calc_eigengap(G_fosr))
    try:
      eigen_gap[4,i] = calc_eigengap(G_gtr)
    except:
      print(calc_eigengap(G_gtr))

    tmd_distance[0,i] = TMD(G_orig_data, G_hhs_data, w=1, L=4)
    tmd_distance[1,i] = TMD(G_orig_data, G_sdrf_data, w=1, L=4)
    tmd_distance[2,i] = TMD(G_orig_data, G_fosr_data, w=1, L=4)
    tmd_distance[3,i] = TMD(G_orig_data, G_gtr_data, w=1, L=4)

    average_commute_times[0,i] = calc_average_commute(G_orig)
    average_commute_times[1,i] = calc_average_commute(G_hhs)
    average_commute_times[2,i] = calc_average_commute(G_sdrf)
    average_commute_times[3,i] = calc_average_commute(G_fosr)
    average_commute_times[4,i] = calc_average_commute(G_gtr)

    average_curvature[0,i] = calc_average_curvature(G_orig)
    average_curvature[1,i] = calc_average_curvature(G_hhs)
    average_curvature[2,i] = calc_average_curvature(G_sdrf)
    average_curvature[3,i] = calc_average_curvature(G_fosr)
    average_curvature[4,i] = calc_average_curvature(G_gtr)

    # print(G_orig.number_of_nodes(), G_hhs.number_of_nodes(), G_sdrf.number_of_nodes(), G_fosr.number_of_nodes())

    Ds = graphs_to_dists([G_orig,G_hhs,G_sdrf,G_fosr,G_gtr])
    dgms = compute_homologies(Ds, 3, sparse = False)
    dists = homology_distance(dgms, metric = "betti")

    bottleneck_distance[0,i] = dists[0,1] #calc_bottleneck_distance(G_orig, G_hhs)
    bottleneck_distance[1,i] = dists[0,2] #calc_bottleneck_distance(G_orig, G_sdrf)
    bottleneck_distance[2,i] = dists[0,3] #calc_bottleneck_distance(G_orig, G_fosr)
    bottleneck_distance[3,i] = dists[0,4] #calc_bottleneck_distance(G_orig, G_gtr)

# Train Networks

In [ ]:
def get_nn(in_channels, out_channels):
  return torch.nn.Sequential(torch.nn.Linear(in_channels, out_channels), torch.nn.ReLU(),
                             torch.nn.Linear(out_channels, out_channels))

class GNN(torch.nn.Module):
  def __init__(self, node_input_dim, num_classes, num_layers, hidden_dim = 128, device = "cuda", arch = "GCN"):
    super().__init__()

    self.num_layers = num_layers
    self.layers = []
    if arch == "GIN":
      self.layers.append(torch_geometric.nn.GINConv(get_nn(node_input_dim, hidden_dim)).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.GINConv(get_nn(hidden_dim, hidden_dim)).to(device))
    elif arch == "Graph":
      self.layers.append(torch_geometric.nn.GraphConv(node_input_dim, hidden_dim).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.GraphConv(hidden_dim, hidden_dim).to(device))
    elif arch == "GAT":
      self.layers.append(torch_geometric.nn.GATConv(node_input_dim, hidden_dim).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.GATConv(hidden_dim, hidden_dim).to(device))
    else:
      self.layers.append(torch_geometric.nn.GCNConv(node_input_dim, hidden_dim).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.GCNConv(hidden_dim, hidden_dim).to(device))

    self.layers = torch.nn.ModuleList(self.layers)
    self.lin = torch.nn.Linear(hidden_dim, num_classes).to(device)

  def forward(self, x, edge_index, batch):
    for i in range(self.num_layers):
      x = self.layers[i](x, edge_index).relu()

    x = torch_geometric.nn.global_mean_pool(x, batch)
    z = self.lin(x)
    return z

In [ ]:
def add_features(dataset, x):
  dataset_new = []
  N = len(dataset)
  for i in tqdm(range(N)):
    data = dataset[i]
    data.x = x[i]
    dataset_new.append(data)
  return dataset_new

def append_features(dataset, x):
  dataset_new = []
  N = len(dataset)
  for i in tqdm(range(N)):
    data = dataset[i]
    data.x = torch.cat((data.x, x[i]), dim=1)
    dataset_new.append(data)
  return dataset_new

def create_features(dataset, dim):
  x = []
  N = len(dataset)
  for i in tqdm(range(N)):
    x.append(torch.randn(dataset[i].num_nodes, dim))
  return x

def create_adj(dataset, method):
  adj = []
  N = len(dataset)
  for i in tqdm(range(N)):
    if method == "empty":
      adj.append(torch.zeros(dataset[i].num_nodes, dataset[i].num_nodes))
    elif method == "full":
      adj.append(torch.ones(dataset[i].num_nodes, dataset[i].num_nodes) - torch.eye(dataset[i].num_nodes))
    else:
      adj.append(to_dense_adj(dataset[i].edge_index)[0])
  return adj

def replace_y(dataset1, dataset2):
  N = len(dataset1)
  for i in tqdm(range(N)):
    dataset1[i].y = dataset2[i].y
  return dataset1

In [ ]:
import torch_geometric.transforms as T
# pre_transform = T.Compose([PrecomputeGTREdges(num_edges=1000)])
# transform = T.Compose([AddPrecomputedGTREdges(num_edges=1000)])
# dataset_gtr = torch_geometric.datasets.TUDataset(root="data/",name = "IMDB-BINARY", transform = transform, pre_transform=pre_transform)

PreGTR = PrecomputeGTREdges(num_edges=3)
data_gtr = []
for i in tqdm(range(len(dataset))):
  G_data = Data(edge_index = dataset[i].edge_index)
  G_data_pre = PreGTR(G_data)
  GTR = AddPrecomputedGTREdges(num_edges=3)
  G_gtr = GTR(copy.deepcopy(G_data_pre)).edge_index
  data_gtr.append(to_dense_adj(G_gtr)[0])

data_gtr = merge_datasets_from_adj(dataset, data_gtr)

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
<class 'networkx.utils.decorators.argmap'> compilation 4:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
100%|██████████| 10000/10000 [00:00<00:00, 28901.76it/s]


In [ ]:
path = "/content/drive/MyDrive/CubeRep-Rewiring/data/"
data_hhs = torch.load(path + "zinc-hhs.pt")
data_sdrf = torch.load(path + "zinc-sdrf.pt")
data_fosr = torch.load(path + "zinc-fosr.pt")

In [ ]:
len(data_hhs)

10000

In [ ]:
len(data_sdrf)

10000

In [ ]:
len(data_fosr)

10000

In [ ]:
data_hhs[0]

Data(x=[29, 1], edge_index=[2, 64], edge_attr=[64], y=[1], rewired_edge_index=[2, 70])

In [ ]:
data_hhs_new = merge_datasets_from_adj(dataset, data_hhs)

100%|██████████| 2000/2000 [00:01<00:00, 1643.09it/s]


In [ ]:
data_empty = merge_datasets_from_adj(dataset, create_adj(dataset, "empty"))
data_full = merge_datasets_from_adj(dataset, create_adj(dataset, "full"))
data_none = merge_datasets_from_adj(dataset, create_adj(dataset, "none"))

100%|██████████| 10000/10000 [00:00<00:00, 29524.61it/s]


In [ ]:
X = create_features(dataset,127)

100%|██████████| 10000/10000 [00:00<00:00, 18697.42it/s]


In [ ]:
data_hhs = append_features(data_hhs, X)
data_fosr = append_features(data_fosr, X)
data_sdrf = append_features(data_sdrf, X)
data_gtr = append_features(data_gtr, X)
data_empty = append_features(data_empty, X)
data_full = append_features(data_full, X)
data_none = append_features(data_none, X)

100%|██████████| 10000/10000 [00:00<00:00, 35032.73it/s]


In [ ]:
data_hhs = replace_y(data_hhs_new, dataset)
data_fosr = replace_y(data_fosr, dataset)
data_sdrf = replace_y(data_sdrf, dataset)
data_gtr = replace_y(data_gtr, dataset)
data_empty = replace_y(data_empty, dataset)
data_full = replace_y(data_full, dataset)
data_none = replace_y(data_none, dataset)

100%|██████████| 10873/10873 [00:00<00:00, 50928.53it/s]


In [ ]:
dataset[3]

Data(x=[27, 1], edge_index=[2, 60], edge_attr=[60], y=[1])

In [ ]:
data_gtr[3]

Data(x=[27, 128], edge_index=[2, 60], edge_attr=[60], y=[1], rewired_edge_index=[2, 66])

In [ ]:
data_none[3]

Data(x=[27, 128], edge_index=[2, 60], edge_attr=[60], y=[1], rewired_edge_index=[2, 60])

In [ ]:
data_empty[3]

Data(x=[27, 128], edge_index=[2, 60], edge_attr=[60], y=[1], rewired_edge_index=[2, 0])

In [ ]:
data_full[3]

Data(x=[27, 128], edge_index=[2, 60], edge_attr=[60], y=[1], rewired_edge_index=[2, 702])

In [ ]:
data_sdrf[3]

Data(x=[27, 128], edge_index=[2, 60], edge_attr=[60], y=[1], rewired_edge_index=[2, 66])

In [ ]:
data_fosr[3]

Data(x=[27, 128], edge_index=[2, 60], edge_attr=[60], y=[1], rewired_edge_index=[2, 66])

In [ ]:
# data_hhs = data_hhs_new
data_hhs[3]

Data(x=[27, 128], edge_index=[2, 60], edge_attr=[60], y=[1], rewired_edge_index=[2, 66])

In [ ]:
for i in range(len(data_hhs)):
  print(data_hhs[i].y.max())

In [ ]:
class RewiredDatasetEdgeIndex(InMemoryDataset):
  def __init__(self, dataset, index = None):
    super(RewiredDatasetEdgeIndex, self).__init__('.', None, None, None)

    self.dataset_rewired = []
    if index == None:
      for i in range(len(dataset)):
        data = dataset[i]
        self.dataset_rewired.append(Data(x = data.x, edge_index = data.rewired_edge_index, y = data.y))
    else:
      for i in range(len(index)):
        self.dataset_rewired.append(dataset[index[i]])

  def get(self, idx):
    return self.dataset_rewired[idx]

  def len(self):
    return len(self.dataset_rewired)

  def _download(self):
    return

  def _process(self):
      return

  def __repr__(self):
      return '{}()'.format(self.__class__.__name__)

In [ ]:
p = torch.randperm(len(dataset))

train_length = int(np.round(0.8*len(dataset)))
val_length = int(np.round(0.9*len(dataset)))

p_train = p[:train_length]
p_val = p[train_length:val_length]
p_test = p[val_length:]

hhs_train = RewiredDatasetEdgeIndex(data_hhs,p_train)
hhs_val = RewiredDatasetEdgeIndex(data_hhs, p_val)
hhs_test = RewiredDatasetEdgeIndex(data_hhs, p_test)

sdrf_train = RewiredDatasetEdgeIndex(data_sdrf,p_train)
sdrf_val = RewiredDatasetEdgeIndex(data_sdrf,p_val)
sdrf_test = RewiredDatasetEdgeIndex(data_sdrf,p_test)

fosr_train = RewiredDatasetEdgeIndex(data_fosr,p_train)
fosr_val = RewiredDatasetEdgeIndex(data_fosr,p_val)
fosr_test = RewiredDatasetEdgeIndex(data_fosr,p_test)

gtr_train = RewiredDatasetEdgeIndex(data_gtr,p_train)
gtr_val = RewiredDatasetEdgeIndex(data_gtr,p_val)
gtr_test = RewiredDatasetEdgeIndex(data_gtr,p_test)

empty_train = RewiredDatasetEdgeIndex(data_empty,p_train)
empty_val = RewiredDatasetEdgeIndex(data_empty,p_val)
empty_test = RewiredDatasetEdgeIndex(data_empty,p_test)

full_train = RewiredDatasetEdgeIndex(data_full,p_train)
full_val = RewiredDatasetEdgeIndex(data_full,p_val)
full_test = RewiredDatasetEdgeIndex(data_full,p_test)

none_train = RewiredDatasetEdgeIndex(data_none,p_train)
none_val = RewiredDatasetEdgeIndex(data_none,p_val)
none_test = RewiredDatasetEdgeIndex(data_none,p_test)

In [ ]:
torch.save((hhs_train, hhs_val, hhs_test),"/content/drive/MyDrive/CubeRep-Rewiring/NNData/Zinc/hhs.pt")
torch.save((gtr_train, gtr_val, gtr_test),"/content/drive/MyDrive/CubeRep-Rewiring/NNData/Zinc/gtr.pt")
torch.save((fosr_train, fosr_val, fosr_test),"/content/drive/MyDrive/CubeRep-Rewiring/NNData/Zinc/fosr.pt")
torch.save((sdrf_train, sdrf_val, sdrf_test),"/content/drive/MyDrive/CubeRep-Rewiring/NNData/Zinc/sdrf.pt")
torch.save((empty_train, empty_val, empty_test),"/content/drive/MyDrive/CubeRep-Rewiring/NNData/Zinc/empty.pt")
torch.save((full_train, full_val, full_test),"/content/drive/MyDrive/CubeRep-Rewiring/NNData/Zinc/full.pt")
torch.save((none_train, none_val, none_test),"/content/drive/MyDrive/CubeRep-Rewiring/NNData/Zinc/none.pt")
torch.save(p, "/content/drive/MyDrive/CubeRep-Rewiring/NNData/Zinc/permutation.pt")

In [ ]:
hhs_train, hhs_val, hhs_test = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/NNData/ESOL/hhs.pt")
gtr_train, gtr_val, gtr_test = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/NNData/ESOL/gtr.pt")
fosr_train, fosr_val, fosr_test = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/NNData/ESOL/fosr.pt")
sdrf_train, sdrf_val, sdrf_test = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/NNData/ESOL/sdrf.pt")
empty_train, empty_val, empty_test = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/NNData/ESOL/empty.pt")
full_train, full_val, full_test = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/NNData/ESOL/full.pt")
none_train, none_val, none_test = torch.load("/content/drive/MyDrive/CubeRep-Rewiring/NNData/ESOL/none.pt")
# torch.save(p, "/content/drive/MyDrive/CubeRep-Rewiring/NNData/PeptidesStruct/permutation.pt")

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
from tqdm import tqdm
from sklearn.metrics import average_precision_score


# model = HHSConvNode(1703,5,128, hidden_dim = 128)
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, weight_decay = 0.01)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000)

path = "./drive/MyDrive/CubeRep-Rewiring/Models/esol/"

def train(model, train_loader, val_loader, test_loader, filename, iterations = 200, criterion = torch.nn.BCEWithLogitsLoss()):
  model.train()

  optimizer = torch.optim.Adam(model.parameters())
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, iterations)

  for i in tqdm(range(iterations)):
    model.train()
    for data in train_loader:
      optimizer.zero_grad()  # Clear gradients.
      data = data.to('cuda')
      out = model(data.x.float(), data.rewired_edge_index, data.batch).flatten() # Perform a single forward pass.
      loss = criterion(out, data.y.float())  # Compute the loss.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
    scheduler.step()
    # if i % 10 == 0:
    #   with torch.no_grad():
    #     print("Trianing Error", loss)
    #     print("Train Acc", test(model, train_loader))
    #     print("Val Acc", test(model, val_loader))
    #     print("Test Acc", test(model, test_loader))

  with torch.no_grad():
    torch.save(model,filename+str(iterations-1)+".pt")


  return model

def test(model, loader, edge_loader = None):
  model.eval()

  correct = 0
  for data in loader:
    data = data.to('cuda')
    out = model(data.x.float(), data.rewired_edge_index, data.batch).flatten()
    pred = (out.sign()/2 + 0.5).round().to(data.y.dtype)  # Use the class with highest probability.
    correct += (pred == data.y).sum()/out.shape[0]
  return correct / (len(loader)) # Derive ratio of correct predictions.

def test_precision(model, loader):
  model.eval()

  correct = 0
  for data in loader:
    data = data.to('cuda')
    out = model(data.x.float(), data.edge_index, data.batch).detach().cpu().sigmoid()
    correct += average_precision_score(data.y.cpu(), out)
  return correct / (len(loader)) # Derive ratio of correct predictions.

def test_mse(model, loader):
  model.eval()

  correct = 0
  for data in loader:
    data = data.to('cuda')
    out = model(data.x.float(), data.rewired_edge_index, data.batch).flatten()
    correct += torch.nn.functional.mse_loss(out,data.y.flatten())
  return correct / (len(loader)) # Derive ratio of correct predictions.

def test_acc(model, loader, edge_loader = None):
  model.eval()

  correct = 0
  for data in loader:
    data = data.to('cuda')
    out = model(data.x.float(), data.edge_index, data.batch).argmax(dim=-1)
    correct += (out == data.y).sum()/out.shape[0]
  return correct / (len(loader)) # Derive ratio of correct predictions.

<IPython.core.display.Javascript object>

In [ ]:
T = 5

path = "./drive/MyDrive/CubeRep-Rewiring/Models/esol/"

layers = [2,3,4]
archs = ["GCN", "Graph", "GAT", "GIN"]


train_errors = torch.zeros(7, len(layers), len(archs), T)
val_errors = torch.zeros(7, len(layers), len(archs), T)
test_errors = torch.zeros(7, len(layers), len(archs), T)

train_loaders = [none_train, full_train, empty_train, hhs_train, fosr_train, gtr_train, sdrf_train]
val_loaders = [none_val, full_val, empty_val, hhs_val, fosr_val, gtr_val, sdrf_val]
test_loaders = [none_test, full_test, empty_test, hhs_test, fosr_test, gtr_test, sdrf_test]

methods = ["none", "full", "empty", "hhs", "fosr", "gtr", "sdrf"]

for i,l in enumerate(layers):
  for j,arch in enumerate(archs):
    for t in range(T):
      for k,method in enumerate(methods):
        train_loader = DataLoader(train_loaders[k], batch_size=100, shuffle=False)
        val_loader = DataLoader(val_loaders[k], batch_size=20, shuffle=False)
        test_loader = DataLoader(test_loaders[k], batch_size=20, shuffle=False)

        # model = GNN(128, 1, l, hidden_dim = 64, arch = arch).to("cuda")
        # model = train(model, train_loader, val_loader, test_loader, path+method+"-"+arch+"-l-"+str(l)+"-t-"+str(t)+"-epoch-", iterations = 100,
                      # criterion = torch.nn.MSELoss())

        model = torch.load(path+method+"-"+arch+"-l-"+str(l)+"-t-"+str(t)+"-epoch-99.pt")


        train_errors[k,i,j,t] = test_mse(model, train_loader).detach().cpu()
        val_errors[k,i,j,t] = test_mse(model, val_loader).detach().cpu()
        test_errors[k,i,j,t] = test_mse(model, test_loader).detach().cpu()

        print(method, " ", arch, " ", train_errors[k,i,j,t],val_errors[k,i,j,t],test_errors[k,i,j,t])


none   GCN   tensor(1.6221) tensor(1.6890) tensor(1.9130)
full   GCN   tensor(1.3928) tensor(1.4990) tensor(1.5465)
empty   GCN   tensor(1.3926) tensor(1.5952) tensor(1.7065)
hhs   GCN   tensor(1.4663) tensor(1.2417) tensor(1.4254)
fosr   GCN   tensor(1.3738) tensor(1.2794) tensor(1.4654)
gtr   GCN   tensor(1.1890) tensor(1.1356) tensor(1.5896)
sdrf   GCN   tensor(1.4299) tensor(1.5098) tensor(1.6752)
none   GCN   tensor(1.5025) tensor(1.4615) tensor(1.6387)
full   GCN   tensor(1.7871) tensor(2.0032) tensor(1.8808)
empty   GCN   tensor(1.3850) tensor(1.5115) tensor(1.6503)
hhs   GCN   tensor(1.3482) tensor(1.1815) tensor(1.4656)
fosr   GCN   tensor(1.4973) tensor(1.4103) tensor(1.5395)
gtr   GCN   tensor(1.4872) tensor(1.4601) tensor(1.7534)
sdrf   GCN   tensor(1.3339) tensor(1.4402) tensor(1.5896)
none   GCN   tensor(1.6188) tensor(1.6113) tensor(1.8911)
full   GCN   tensor(1.6103) tensor(1.8606) tensor(1.8136)
empty   GCN   tensor(1.5026) tensor(1.5746) tensor(1.8294)
hhs   GCN   ten

In [ ]:
path = "./drive/MyDrive/CubeRep-Rewiring/Models/esol/"
torch.save((train_errors, val_errors, test_errors), path+"acc.pt")

In [ ]:
T = 5

path = "./drive/MyDrive/CubeRep-Rewiring/Models/PeptidesFunc/"

layers = [2,3,4]
archs = ["GCN", "Graph", "GAT", "GIN"]


train_errors = torch.zeros(7, len(layers), len(archs), T)
val_errors = torch.zeros(7, len(layers), len(archs), T)
test_errors = torch.zeros(7, len(layers), len(archs), T)

train_loaders = [none_train, empty_train, hhs_train, fosr_train, gtr_train, sdrf_train]
val_loaders = [none_val, empty_val, hhs_val, fosr_val, gtr_val, sdrf_val]
test_loaders = [none_test, empty_test, hhs_test, fosr_test, gtr_test, sdrf_test]

methods = ["none", "empty", "hhs", "fosr", "gtr", "sdrf"]

for i,l in enumerate(layers):
  for j,arch in enumerate(archs):
    for t in range(T):
      for k,method in enumerate(methods):
        train_loader = DataLoader(train_loaders[k], batch_size=50, shuffle=False)
        val_loader = DataLoader(val_loaders[k], batch_size=20, shuffle=False)
        test_loader = DataLoader(test_loaders[k], batch_size=20, shuffle=False)

        model = GNN(9, 10, l, hidden_dim = 64, arch = arch).to("cuda")
        model = train(model, train_loader, val_loader, test_loader, path+method+"-mse-"+arch+"-"+str(l)+"-epoch-", iterations = 200,
                      criterion = torch.nn.MSELoss())
        # model = torch.load(path+method+"-"+arch+"-"+str(l)+"-epoch-"+str(99)+".pt")

        train_errors[k,i,j,t] = test_precision(model, train_loader)
        val_errors[k,i,j,t] = test_precision(model, val_loader)
        test_errors[k,i,j,t] = test_precision(model, test_loader)

        print(method, " ", arch, " ", train_errors[k,i,j,t],val_errors[k,i,j,t],test_errors[k,i,j,t])


100%|██████████| 200/200 [04:39<00:00,  1.40s/it]


none   GCN   tensor(0.4628) tensor(0.4390) tensor(0.4415)


100%|██████████| 200/200 [04:21<00:00,  1.31s/it]


empty   GCN   tensor(0.3435) tensor(0.3531) tensor(0.3544)


100%|██████████| 200/200 [04:38<00:00,  1.39s/it]


...

gtr   GCN   tensor(0.4010) tensor(0.3998) tensor(0.3897)


100%|██████████| 200/200 [04:36<00:00,  1.38s/it]


sdrf   GCN   tensor(0.4556) tensor(0.4342) tensor(0.4359)


 14%|█▍        | 29/200 [00:40<03:58,  1.39s/it]


KeyboardInterrupt: 

In [ ]:
path = "./drive/MyDrive/CubeRep-Rewiring/Models/esol/"
torch.save((train_errors, val_errors, test_errors), path+"acc.pt")

NameError: name 'torch' is not defined

In [ ]:
def print_diff(test, i,j):
  return test[i,:,:,:].mean(dim=-1) - test[j,:,:,:].mean(dim=-1)

In [ ]:
x = torch.randn(5)
x

tensor([ 1.7464, -0.0064, -1.0812,  0.1173, -1.5771])

In [ ]:
sorted, indices = torch.sort(x)
sorted

tensor([-1.5771, -1.0812, -0.0064,  0.1173,  1.7464])

In [ ]:
indices

tensor([4, 2, 1, 3, 0])

In [ ]:
test_errors.shape

torch.Size([7, 3, 4, 5])

In [ ]:
def mean_rank(test, desc = True):
  num_methods = test.shape[0]
  ranks = torch.zeros_like(test)
  _, idx = torch.sort(test, descending=desc)
  for k in range(num_methods):
    method_idx = idx[k]
    ranks[method_idx] = k
  return ranks

In [ ]:
def best_val_max(val,test):
  K,N,M,T = test.shape
  best_test = torch.zeros(K)
  for k in range(K):
    max = val[k,:,:,:].min()
    idxes = []
    for j in range(M):
      for t in range(T):
        for i in range(N):
          if (val[k,i,j,t] - max).abs() < 1e-5:
            idxes.append((i,j,t))
    for i,j,t in idxes:
      best_test[k] += test_errors[k,i,j,t]/len(idxes)
  return best_test

def best_val_min(val,test):
  K,N,M,T = test.shape
  best_test = torch.zeros(K)
  for k in range(K):
    min = val[k,:,:,:].min()
    idxes = []
    for j in range(M):
      for t in range(T):
        for i in range(N):
          if (val[k,i,j,t] - min).abs() < 1e-5:
            idxes.append((i,j,t))
    for i,j,t in idxes:
      best_test[k] += test_errors[k,i,j,t]/len(idxes)
  return best_test

def best_val(val,test,max):
  if max:
    return best_val_max(val,test)
  else:
    return best_val_min(val,test)

In [ ]:
path = "./drive/MyDrive/CubeRep-Rewiring/Models/PeptidesStruct/"
train_errors, val_errors, test_errors = torch.load(path+"acc.pt")

In [ ]:
datasets = ["PeptidesStruct", "Lipo", "BACE", "esol", "Zinc", "IMDB/GCN", "Reddit", "Collab", "Mutag", "Proteins", "Enzymes"]
descending = [False, False, False, False, False, True, True, True, True, True, True]

mean_ranks = torch.zeros(len(datasets),7)

for i in range(len(datasets)):
  path = "./drive/MyDrive/CubeRep-Rewiring/Models/"+datasets[i]
  train_errors, val_errors, test_errors = torch.load(path+"/acc.pt")
  if test_errors[6,:,:,:].mean() < 1e-5:
    print(datasets[i])
    test_errors[6,:,:,:] = test_errors[5,:,:,:] # Move SDRF
    test_errors[5,:,:,:] = test_errors[4,:,:,:] # Move GTR
    test_errors[4,:,:,:] = test_errors[3,:,:,:] # Move FOSR
    test_errors[3,:,:,:] = test_errors[2,:,:,:] # Move RelWire
    test_errors[2,:,:,:] = test_errors[1,:,:,:] # Move Empty

    val_errors[6,:,:,:] = val_errors[5,:,:,:] # Move SDRF
    val_errors[5,:,:,:] = val_errors[4,:,:,:] # Move GTR
    val_errors[4,:,:,:] = val_errors[3,:,:,:] # Move FOSR
    val_errors[3,:,:,:] = val_errors[2,:,:,:] # Move RelWire
    val_errors[2,:,:,:] = val_errors[1,:,:,:] # Move Empty
    if i == 0:
      val_errors[1,:,:,:] = 100
      test_errors[1,:,:,:] = 100
    else:
      val_errors[1,:,:,:] = 0
      test_errors[1,:,:,:] = 0
  desc = descending[i]
  best_test_error = best_val(val_errors, test_errors, desc)
  mean_ranks[i,:] = mean_rank(best_test_error, desc)

PeptidesStruct
Reddit
Collab


In [ ]:
import torch
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mean_ranks = torch.load("./drive/MyDrive/CubeRep-Rewiring/Models/ranks.pt")

In [ ]:
mean_ranks.shape

torch.Size([11, 7, 4])

In [ ]:
means = mean_ranks+1
means

tensor([[5., 7., 6., 3., 2., 1., 4.],
        [7., 1., 5., 3., 4., 6., 2.],
        [1., 6., 7., 2., 4., 3., 5.],
        [3., 5., 7., 2., 1., 4., 6.],
        [3., 6., 7., 5., 4., 2., 1.],
        [6., 1., 5., 3., 2., 7., 4.],
        [1., 7., 2., 3., 6., 5., 4.],
        [2., 7., 6., 5., 4., 3., 1.],
        [2., 3., 4., 5., 1., 6., 7.],
        [2., 7., 6., 3., 5., 1., 4.],
        [3., 7., 6., 1., 4., 5., 2.]])

In [ ]:
means.mean(dim=0)

tensor([3.1818, 5.1818, 5.5455, 3.1818, 3.3636, 3.9091, 3.6364])

In [ ]:
stds = mean_ranks.std(dim=[-1])/2
stds

tensor([[0.0000, 0.0000, 0.0000, 0.2500, 0.2500, 0.0000, 0.0000],
        [0.7071, 0.2887, 1.3150, 0.8539, 0.8660, 0.7071, 0.7071],
        [0.2500, 0.2500, 0.0000, 0.8539, 0.4787, 0.8539, 0.4787],
        [1.2500, 0.8539, 0.2500, 1.1902, 0.5000, 0.7071, 0.4787],
        [0.7500, 0.5000, 0.2500, 0.2500, 0.2500, 0.5000, 0.7071],
        [0.9465, 0.7071, 1.3769, 0.4082, 0.8539, 1.1087, 0.6455],
        [0.8539, 0.0000, 0.0000, 0.6455, 0.8539, 0.6455, 0.9129],
        [0.7500, 0.0000, 0.0000, 0.2500, 1.1547, 0.4787, 0.6292],
        [1.0308, 1.7321, 1.0000, 1.0308, 0.7500, 0.8539, 0.5000],
        [0.7500, 0.0000, 0.0000, 0.7500, 0.4082, 0.8539, 0.8539],
        [1.5000, 0.2887, 0.4787, 0.7071, 0.7071, 0.6292, 0.6455]])

In [ ]:
stds = mean_ranks.std(dim=[-1,-2])/np.sqrt(12)

In [ ]:
archs = ["GCN", "GraphConv", "GAT", "GIN"]
titles = ["Dataset", "None", "Full", "Empty", "RelWire", "FOSR", "GTR", "SDRF"]
datasets = ["Peptides", "Lipo", "BACE", "ESOL", "Zinc", "IMDB", "Reddit", "Collab", "Mutag", "Proteins", "Enzymes"]


table = "\\toprule "

for i in range(5):
  for j in range(8):
    if i == 0:
      if j == 0:
        table = table + "Archiecture & "
      elif j < 7:
        table = table + titles[j] + " & "
      else:
        table = table + titles[j] + " \\\\ \midrule \n"
    else:
      if j == 0:
        table = table + archs[i-1] + " & "
      elif j < 7:
        table = table + "$ "+f"{means[i-1,j-1].item():.2f}"+" \pm "+f"{stds[i-1,j-1].item():.2f}"+" $ & "
      else:
        table = table + "$ "+f"{means[i-1,j-1].item():.2f}"+" \pm "+f"{stds[i-1,j-1].item():.2f}"+" $ \\\\ \n"
table = table + "\\bottomrule "
table

'\\toprule Archiecture & None & Full & Empty & RelWire & FOSR & GTR & SDRF \\\\ \\midrule \nGCN & $ 3.58 \\pm 0.35 $ & $ 4.79 \\pm 0.44 $ & $ 5.73 \\pm 0.24 $ & $ 3.12 \\pm 0.28 $ & $ 3.97 \\pm 0.28 $ & $ 3.21 \\pm 0.28 $ & $ 3.61 \\pm 0.31 $ \\\\ \nGraphConv & $ 3.42 \\pm 0.35 $ & $ 5.91 \\pm 0.34 $ & $ 5.42 \\pm 0.34 $ & $ 3.36 \\pm 0.27 $ & $ 3.36 \\pm 0.26 $ & $ 2.91 \\pm 0.27 $ & $ 3.61 \\pm 0.25 $ \\\\ \nGAT & $ 3.24 \\pm 0.33 $ & $ 4.88 \\pm 0.42 $ & $ 5.09 \\pm 0.33 $ & $ 4.03 \\pm 0.35 $ & $ 3.45 \\pm 0.30 $ & $ 3.48 \\pm 0.27 $ & $ 3.82 \\pm 0.31 $ \\\\ \nGIN & $ 3.36 \\pm 0.36 $ & $ 5.18 \\pm 0.43 $ & $ 5.58 \\pm 0.26 $ & $ 3.42 \\pm 0.24 $ & $ 3.39 \\pm 0.28 $ & $ 3.06 \\pm 0.31 $ & $ 4.00 \\pm 0.27 $ \\\\ \n\\bottomrule '

In [ ]:
print(table)

\toprule Archiecture & None & Full & Empty & RelWire & FOSR & GTR & SDRF \\ \midrule 
GCN & $ 3.58 \pm 0.35 $ & $ 4.79 \pm 0.44 $ & $ 5.73 \pm 0.24 $ & $ 3.12 \pm 0.28 $ & $ 3.97 \pm 0.28 $ & $ 3.21 \pm 0.28 $ & $ 3.61 \pm 0.31 $ \\ 
GraphConv & $ 3.42 \pm 0.35 $ & $ 5.91 \pm 0.34 $ & $ 5.42 \pm 0.34 $ & $ 3.36 \pm 0.27 $ & $ 3.36 \pm 0.26 $ & $ 2.91 \pm 0.27 $ & $ 3.61 \pm 0.25 $ \\ 
GAT & $ 3.24 \pm 0.33 $ & $ 4.88 \pm 0.42 $ & $ 5.09 \pm 0.33 $ & $ 4.03 \pm 0.35 $ & $ 3.45 \pm 0.30 $ & $ 3.48 \pm 0.27 $ & $ 3.82 \pm 0.31 $ \\ 
GIN & $ 3.36 \pm 0.36 $ & $ 5.18 \pm 0.43 $ & $ 5.58 \pm 0.26 $ & $ 3.42 \pm 0.24 $ & $ 3.39 \pm 0.28 $ & $ 3.06 \pm 0.31 $ & $ 4.00 \pm 0.27 $ \\ 
\bottomrule 


In [ ]:
titles = ["Dataset", "None", "Full", "Empty", "RelWire", "FOSR", "GTR", "SDRF"]

table = "\\toprule "

for i in range(12):
  for j in range(8):
    if i == 0:
      table = table + titles[j]
      if j < 7:
        table = table + " & "
      else:
        table = table + " \\\\ \midrule \n"
    else:
      if j == 0:
        table = table + datasets[i-1] + " & "
      elif j < 7:
        table = table + "$ "+f"{means[i-1,j-1].item():.2f}"+" \pm "+f"{stds[i-1,j-1].item():.2f}"+" $ & "
      else:
        table = table + "$ "+f"{means[i-1,j-1].item():.2f}"+" \pm "+f"{stds[i-1,j-1].item():.2f}"+" $ \\\\ \n"
table = table + "\\bottomrule "
table

'\\toprule Dataset & None & Full & Empty & RelWire & FOSR & GTR & SDRF \\\\ \\midrule \nPeptidesStruct & $ 4.00 \\pm 0.12 $ & $ 6.00 \\pm 0.00 $ & $ 4.92 \\pm 0.08 $ & $ 1.58 \\pm 0.15 $ & $ 1.42 \\pm 0.15 $ & $ 0.00 \\pm 0.00 $ & $ 3.08 \\pm 0.08 $ \\\\ \nLipo & $ 4.92 \\pm 0.45 $ & $ 0.67 \\pm 0.50 $ & $ 2.08 \\pm 0.62 $ & $ 4.00 \\pm 0.43 $ & $ 2.58 \\pm 0.45 $ & $ 3.67 \\pm 0.31 $ & $ 3.08 \\pm 0.42 $ \\\\ \nBACE & $ 0.92 \\pm 0.29 $ & $ 5.42 \\pm 0.19 $ & $ 5.50 \\pm 0.15 $ & $ 2.83 \\pm 0.44 $ & $ 2.17 \\pm 0.32 $ & $ 1.58 \\pm 0.45 $ & $ 2.58 \\pm 0.38 $ \\\\ \nesol & $ 2.58 \\pm 0.63 $ & $ 3.33 \\pm 0.64 $ & $ 5.17 \\pm 0.21 $ & $ 2.58 \\pm 0.50 $ & $ 1.50 \\pm 0.50 $ & $ 1.83 \\pm 0.37 $ & $ 4.00 \\pm 0.41 $ \\\\ \nZinc & $ 1.25 \\pm 0.39 $ & $ 5.67 \\pm 0.14 $ & $ 5.25 \\pm 0.18 $ & $ 3.25 \\pm 0.39 $ & $ 2.08 \\pm 0.29 $ & $ 2.25 \\pm 0.30 $ & $ 1.25 \\pm 0.46 $ \\\\ \nIMDB/GCN & $ 3.25 \\pm 0.57 $ & $ 3.25 \\pm 0.59 $ & $ 3.58 \\pm 0.58 $ & $ 1.67 \\pm 0.50 $ & $ 4.25 \\pm 

In [ ]:
print(table)

\toprule Dataset & None & Full & Empty & RelWire & FOSR & GTR & SDRF \\ \midrule 
PeptidesStruct & $ 4.00 \pm 0.12 $ & $ 6.00 \pm 0.00 $ & $ 4.92 \pm 0.08 $ & $ 1.58 \pm 0.15 $ & $ 1.42 \pm 0.15 $ & $ 0.00 \pm 0.00 $ & $ 3.08 \pm 0.08 $ \\ 
Lipo & $ 4.92 \pm 0.45 $ & $ 0.67 \pm 0.50 $ & $ 2.08 \pm 0.62 $ & $ 4.00 \pm 0.43 $ & $ 2.58 \pm 0.45 $ & $ 3.67 \pm 0.31 $ & $ 3.08 \pm 0.42 $ \\ 
BACE & $ 0.92 \pm 0.29 $ & $ 5.42 \pm 0.19 $ & $ 5.50 \pm 0.15 $ & $ 2.83 \pm 0.44 $ & $ 2.17 \pm 0.32 $ & $ 1.58 \pm 0.45 $ & $ 2.58 \pm 0.38 $ \\ 
esol & $ 2.58 \pm 0.63 $ & $ 3.33 \pm 0.64 $ & $ 5.17 \pm 0.21 $ & $ 2.58 \pm 0.50 $ & $ 1.50 \pm 0.50 $ & $ 1.83 \pm 0.37 $ & $ 4.00 \pm 0.41 $ \\ 
Zinc & $ 1.25 \pm 0.39 $ & $ 5.67 \pm 0.14 $ & $ 5.25 \pm 0.18 $ & $ 3.25 \pm 0.39 $ & $ 2.08 \pm 0.29 $ & $ 2.25 \pm 0.30 $ & $ 1.25 \pm 0.46 $ \\ 
IMDB/GCN & $ 3.25 \pm 0.57 $ & $ 3.25 \pm 0.59 $ & $ 3.58 \pm 0.58 $ & $ 1.67 \pm 0.50 $ & $ 4.25 \pm 0.46 $ & $ 2.75 \pm 0.69 $ & $ 2.25 \pm 0.45 $ \\ 
Reddit & $

In [ ]:
mean_ranks.mean(dim=0)

tensor([2.4015, 4.1894, 4.4545, 2.4848, 2.5455, 2.1667, 2.7576])

In [ ]:
datasets = ["PeptidesStruct", "Lipo", "BACE", "esol", "Zinc", "IMDB/GCN", "Reddit", "Collab", "Mutag", "Proteins", "Enzymes"]
descending = [False, False, False, False, False, True, True, True, True, True, True]

mean_ranks_2 = torch.zeros(len(datasets), 7)

for i in range(len(datasets)):
  path = "./drive/MyDrive/CubeRep-Rewiring/Models/"+datasets[i]
  train_errors, val_errors, test_errors = torch.load(path+"/acc.pt")
  if test_errors[6,:,:,:].mean() < 1e-5:
    test_errors[6,:,:,:] = test_errors[5,:,:,:] # Move SDRF
    test_errors[5,:,:,:] = test_errors[4,:,:,:] # Move GTR
    test_errors[4,:,:,:] = test_errors[3,:,:,:] # Move FOSR
    test_errors[3,:,:,:] = test_errors[2,:,:,:] # Move RelWire
    test_errors[2,:,:,:] = test_errors[1,:,:,:] # Move Empty
    if i == 0:
      test_errors[1,:,:,:] = 100
    else:
      test_errors[1,:,:,:] = 0
  desc = descending[i]
  mean_ranks_2[i,:] = mean_rank(test_errors.mean(dim=-1), desc).mean(dim=[1,2])

In [ ]:
mean_ranks_2

tensor([[4.0833, 6.0000, 4.9167, 1.4167, 1.6667, 0.0833, 2.8333],
        [5.0000, 1.6667, 1.7500, 3.8333, 2.4167, 2.7500, 3.5833],
        [0.5833, 5.5000, 5.4167, 3.1667, 2.5000, 1.3333, 2.5000],
        [2.6667, 4.0000, 4.5833, 2.3333, 1.7500, 1.6667, 4.0000],
        [0.6667, 5.7500, 5.2500, 2.9167, 1.7500, 3.1667, 1.5000],
        [2.4167, 3.5000, 3.1667, 1.6667, 3.9167, 2.4167, 3.9167],
        [1.8333, 6.0000, 5.0000, 2.3333, 1.8333, 1.5833, 2.4167],
        [1.6667, 6.0000, 5.0000, 1.8333, 2.2500, 2.4167, 1.8333],
        [1.4167, 4.0833, 1.6667, 3.5000, 3.9167, 2.5833, 3.8333],
        [3.7500, 1.0833, 6.0000, 1.7500, 3.0833, 2.1667, 3.1667],
        [0.5833, 5.7500, 4.5833, 2.5833, 3.2500, 3.0000, 1.2500]])

In [ ]:
mean_ranks_2.mean(dim=0)

tensor([2.2424, 4.4848, 4.3030, 2.4848, 2.5758, 2.1061, 2.8030])

In [ ]:
test_errors.mean(dim=[1,2,3])

tensor([1.8758e+00, 2.3118e+04, 1.6993e+00, 1.8546e+00, 1.8031e+00, 1.8238e+00,
        1.8445e+00])

In [ ]:
print_diff(test_errors,2,0) # None

tensor([[-0.1324, -0.1183, -0.0939, -0.0991],
        [-0.0757, -0.1013, -0.0985, -0.0898],
        [-0.0708, -0.0889, -0.0857, -0.0856]])

In [ ]:
print_diff(test_errors,2,1) # Full

tensor([[-0.1110, -0.2818, -0.1832, -0.1660],
        [-0.1074, -0.1986, -0.1456, -0.1961],
        [-0.1110, -0.1823, -0.1384, -0.2362]])

In [ ]:
print_diff(test_errors,3,2) # Empty

tensor([[ 0.0443, -0.0148,  0.0554, -0.0170],
        [ 0.0626, -0.0245,  0.0307, -0.0239],
        [ 0.0201, -0.0320,  0.0025, -0.0249]])

In [ ]:
print_diff(test_errors,2,3) # FOSR

tensor([[-0.0443,  0.0148, -0.0554,  0.0170],
        [-0.0626,  0.0245, -0.0307,  0.0239],
        [-0.0201,  0.0320, -0.0025,  0.0249]])

In [ ]:
print_diff(test_errors,2,4) # GTR

tensor([[ 0.0086,  0.0844, -0.0220,  0.0763],
        [ 0.0405,  0.0832,  0.0197,  0.0877],
        [ 0.0481,  0.0896,  0.0291,  0.0821]])

In [ ]:
print_diff(test_errors,2,5) # SDRF

tensor([[-0.0420, -0.0634, -0.0750, -0.0490],
        [-0.0725, -0.0484, -0.0872, -0.0407],
        [-0.0169, -0.0390, -0.0324, -0.0365]])

In [ ]:
test_errors[3,:,:,:].mean(dim=-1)

tensor([[0.2393, 0.2159, 0.2270, 0.2286],
        [0.2334, 0.1902, 0.2203, 0.2226],
        [0.2293, 0.1891, 0.2148, 0.2002]])